<a href="https://colab.research.google.com/github/stefkong1982/netology.ru/blob/Master/%D0%91%D0%B8%D0%B1%D0%BB%D0%B8%D0%BE%D1%82%D0%B5%D0%BA%D0%B8%20Python%20%D0%B4%D0%BB%D1%8F%20%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85/%D0%9B%D0%B5%D0%BA%D1%86%D0%B8%D1%8F%20%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B%20%D0%B2%D0%B5%D0%B1-%D1%81%D0%BA%D1%80%D0%B0%D0%BF%D0%B8%D0%BD%D0%B3%D0%B0/%D0%94%D0%97_%D0%9A%D0%BE%D0%BD%D0%B4%D1%80%D0%B0%D1%82%D1%8C%D0%B5%D0%B2_NUMPY-48%20%D0%BA%20%D0%BB%D0%B5%D0%BA%D1%86%D0%B8%D0%B8%20%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B%20%D0%B2%D0%B5%D0%B1-%D1%81%D0%BA%D1%80%D0%B0%D0%BF%D0%B8%D0%BD%D0%B3%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание к лекции "Основы веб-скрапинга"

## Обязательная часть

Вам необходимо написать функцию, которая будет основана на **поиске** по сайту [habr.com](https://habr.com/ru/search/).
Функция в качестве параметра должна принимать **список** запросов для поиска (например, `['python', 'анализ данных']`) и на основе материалов, попавших в результаты поиска по **каждому** запросу, возвращать датафрейм вида:

```
<дата> - <заголовок> - <ссылка на материал>
```

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import urllib.parse

base_url = 'https://habr.com'
res = requests.get('https://habr.com/ru/search/?q=анализ%20данных&target_type=posts&order=relevance')
soup = BeautifulSoup(res.text)
news = soup.find_all('article', class_='tm-articles-list__item')

for article in news:
    # Проверка наличия заголовка в разметке megaposts
    title_element = article.find('h2', class_='tm-megapost-snippet__title')
    if title_element is not None:
        title = title_element.text.strip()
        print("Заголовок:", title)
    else:
        # Проверка наличия заголовка в другой разметке
        title_element_alt = article.find('h2', class_='tm-title tm-title_h2')
        if title_element_alt is not None:
            title = title_element_alt.find('span').text.strip()
            print("Заголовок:", title)
        else:
            print("Заголовок не найден")

    # Проверка наличия ссылки в разметке megaposts
    link_element = article.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__date')
    if link_element is not None:
        relative_link = link_element.get('href')
        full_link = urllib.parse.urljoin(base_url, relative_link)
        print(full_link)
    else:
        # Проверка наличия ссылки в другой разметке
        link_element_alt = article.find('h2', class_='tm-title tm-title_h2').find('a', class_='tm-title__link')
        if link_element_alt is not None:
            relative_link = link_element_alt.get('href')
            full_link = urllib.parse.urljoin(base_url, relative_link)
            print(full_link)
        else:
            print("Ссылка не найдена")

    # Проверка наличия даты в разметке megaposts
    date_element = article.find('time', class_='tm-megapost-snippet__datetime-published')
    if date_element is not None:
        date = date_element.get('title')
        print("Дата:", date)
    else:
        # Проверка наличия даты в другой разметке
        date_element_alt = article.find('span', class_='tm-article-datetime-published').find('time')
        if date_element_alt is not None:
            date = date_element_alt.get('title')
            print("Дата:", date)
        else:
            print("Дата не найдена")

    # Проверка наличия полного текста в разметке megaposts
    content_element = article.find('div', id='post-content-body')
    if content_element is not None:
        content = content_element.text.strip()
        print("Полный текст:", content)
    else:
        # Проверка наличия полного текста в другой разметке
        content_element_alt = article.find('div', class_='tm-article-presenter__body')
        if content_element_alt is not None:
            content = content_element_alt.text.strip()
            print("Полный текст:", content)
        else:
            print("Полный текст не найден")

    print()

Заголовок: Пять лет Школе анализа данных
https://habr.com/ru/companies/yandex/articles/148443/
Дата: 2012-07-25, 13:29
Полный текст не найден

Заголовок: 10 лет Школе анализа данных Яндекса
https://habr.com/ru/companies/yandex/articles/334066/
Дата: 2017-07-25, 13:47
Полный текст не найден

Заголовок: Как за две недели проверить гипотезы применимости анализа данных в горно-металлургической компании
https://habr.com/ru/companies/factory5/articles/681846/
Дата: 2022-08-10, 15:52
Полный текст не найден

Заголовок: Что такое бессерверный SQL? И как использовать его для анализа данных?
https://habr.com/ru/companies/microsoft/articles/537064/
Дата: 2021-01-14, 10:00
Полный текст не найден

Заголовок: Как мы строим систему обработки, хранения и анализа данных в СИБУРе
https://habr.com/ru/companies/sibur_official/articles/436632/
Дата: 2019-01-21, 11:21
Полный текст не найден

Заголовок: Становясь Пангеей: будущее современного стека для анализа данных
https://habr.com/ru/articles/763942/
Дата:

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse
import time

def get_habr_posts(query):
    """
    Получает заголовки и ссылки на статьи с сайта habr.com по заданному запросу.

    Аргументы:
    query -- список запросов для поиска статей (например, ['python', 'анализ данных'])

    Возвращает:
    DataFrame -- датафрейм с информацией о заголовках и ссылках на статьи
    """

    habr_blog = pd.DataFrame()
    base_url = "https://habr.com"

    for q in query:
        URL = 'https://habr.com/ru/search/'
        params = {
            'q': q
        }
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text)
        articles = soup.find_all('article', class_='tm-articles-list__item')

        if not articles:
            continue

        posts = []
        for article in articles:
            title_element = article.find('h2', class_='tm-megapost-snippet__title')
            if title_element is not None:
                title = title_element.text.strip()
                link_element = article.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__date')
                relative_link = link_element.get('href')
                full_link = urllib.parse.urljoin(base_url, relative_link)
                date_element = article.find('time', class_='tm-megapost-snippet__datetime-published')
                date = date_element.get('title')
            else:
                title_element_alt = article.find('h2', class_='tm-title tm-title_h2')
                title = title_element_alt.find('span').text.strip()
                link_element_alt = article.find('h2', class_='tm-title tm-title_h2').find('a', class_='tm-title__link')
                relative_link = link_element_alt.get('href')
                full_link = urllib.parse.urljoin(base_url, relative_link)
                date_element_alt = article.find('span', class_='tm-article-datetime-published').find('time')
                date = date_element_alt.get('title')

            post = {'date': date, 'title': title, 'link': full_link}
            posts.append(post)


        habr_blog = pd.concat([habr_blog, pd.DataFrame(posts)], ignore_index=True)

    # Удаление дубликатов
    habr_blog = habr_blog.drop_duplicates()

    return habr_blog

# Пример использования функции
res = get_habr_posts(['python', 'анализ данных'])
res

,date,title,link
0,"2023-11-01, 18:41",Какую библиотеку на Python выбрать для создани...,https://habr.com/ru/companies/otus/articles/77...
1,"2023-10-29, 12:40",Искусственный интеллект на Python с использова...,https://habr.com/ru/articles/770554/
2,"2023-07-25, 20:02",Python декораторы на максималках. Универсальны...,https://habr.com/ru/articles/750312/
3,"2023-10-28, 15:43",Разработка real-time приложений с Python и Web...,https://habr.com/ru/companies/otus/articles/77...
4,"2023-10-26, 20:57",“Ну и долго мне ещё до магазина?” Или пара сло...,https://habr.com/ru/articles/770216/
5,"2023-10-31, 15:58",Как работают Model Serving инструменты изнутри...,https://habr.com/ru/articles/770886/
6,"2023-09-19, 07:42",Microsoft представила дополнение Python Editor...,https://habr.com/ru/news/761862/
7,"2020-04-21, 18:35","Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/498364/
8,"2023-08-22, 20:14",Microsoft добавила Python в Excel,https://habr.com/ru/news/756266/
9,"2023-10-09, 08:39",JetBrains и Python Software Foundation рассказ...,https://habr.com/ru/news/766124/


In [8]:
def add_full_text(posts_df):
    i = 0
    for el in posts_df['link']:
        # print(el)
        req = requests.get(el).text
        soup = BeautifulSoup(req)
        time.sleep(0.3)
        full_text = soup.find('div', class_='tm-article-presenter__body').text.strip()
        posts_df.loc[i, 'text'] = full_text
        i += 1
    return posts_df

add_full_text(res)

AttributeError: ignored

## Дополнительная часть (необязательная)

Функция из обязательной части задания должна быть расширена следующим образом:
- кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента `4` необходимо получить материалы с первых 4 страниц результатов;
- в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
```
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>
```
